In [1]:
import sys
import os
import pandas as pd
import numpy as np
import re


def read_file(filename):
    file = open(filename, mode='r', encoding='utf-8')
    data = file.read()
    file.close()   
    return data

def make_dataframe(data):
    messages = re.findall('(\d+/\d+/\d+, \d+:\d+:\d+ [A-Z]*]) (.*?:) (.*?\[)', data, flags=re.DOTALL)
    temp_df = pd.DataFrame(messages, columns=['date', 'actor_id', 'message_text'])
    return temp_df


folder_path = './Chats'
df = pd.DataFrame(columns=['date', 'actor_id', 'message_text', 'chat group name'])

for f in os.listdir(folder_path):
    f_path = folder_path + '/' + f
    data = read_file(f_path)
    temp_df = make_dataframe(data)
    temp_df['chat group name'] = f
    print(temp_df['actor_id'][0])
    df = df.append(temp_df)
    
print(df.shape)
df.head()

Social media marketing 20:
Tech Business group:
Social media marketing 3:
Rock Bikers🥏:
Ivr leads available:
Lost premium Support:
👩‍🎤hamari adhuri kahani:
10. Ap group Rahul Singh☕:
Australia Bank Loading :
Corona worriors, 💂🏻‍♂️:
Cards Blocking & Refund:
blackscreen remote softwa:
Website vala:
(43307, 4)


,date,actor_id,message_text,chat group name
0,"11/6/19, 11:31:28 AM]",Social media marketing 20:,‎Messages and calls are end-to-end encrypted. ...,chat_12.txt
1,"11/6/19, 11:31:28 AM]",‎‪+91 98907 76732‬ created this group\n[11/8/1...,*🥳Next week advance prediction video now avail...,chat_12.txt
2,"11/9/19, 10:55:40 AM]",‪+234 701 028 3792‬:,```Is there any loader in this groups in need ...,chat_12.txt
3,"11/9/19, 11:53:46 AM]",‪+91 90413 82638‬:,*ZEFTRON SOLUTIONS*\n\n *INSTANT PAYOUT*\n\n *...,chat_12.txt
4,"11/9/19, 12:01:33 PM]",‪+255 745 032 469‬:,‎image omitted\n[,chat_12.txt


In [2]:
df['message_text'] = df['message_text'].map(lambda x: x.lower())
df['message_text'] = df['message_text'].map(lambda x: x.replace('\n', ' '))
df['message_text'] = df['message_text'].map(lambda x: x.replace('\\', ' '))

In [3]:
print(pd.pivot_table(df, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(df, index=['actor_id'], aggfunc=pd.Series.count).shape)

(13538, 3)
(3158, 3)


In [4]:
import re
import phonenumbers

def has_numbers(input_string):
    numbers = re.findall('(\d)', input_string)
    if len(numbers) == 0:
        return False
    
    if len(numbers) == 1: 
        if phonenumbers.PhoneNumberMatcher(input_string, 'US'):
            return False
    
    return True
    

df['has_digit'] = df['message_text'].apply(has_numbers) 
df[df['has_digit'] == True].shape

(23028, 5)

In [5]:
def empty_post(msg):
    if msg == 'na':
         return True
    return False

df['empty_post'] = df['message_text'].apply(empty_post)
df[df['empty_post'] == True].shape

(0, 6)

In [6]:
def money_laundering(msg):
    
    available = 'available' in msg
    payment = 'payment' in msg or 'payout' in msg
    blocking = 'blocking' in msg
    
    giftcard = 'gift card' in msg or 'giftcard' in msg or 'gifts card' in msg 
    if giftcard and (blocking or available or payment):
        return 'giftcard'
    
    
    gateway = 'gatewy' in msg or 'gateway' in msg or 'gatway' in msg
    payment_form = gateway or payment or 'loader' in msg or 'merchant' in msg or 'link' in msg
    two_d = '2d' in msg or '2-d' in msg or '2 d' in msg or '3d' in msg or '3-d' in msg or '3 d' in msg or 'e check' in msg
    if two_d and payment_form:
        return '2d or 3d gateway'
    
    
    bank = 'bank account' in msg
    if bank:
        return 'bank account'
    
    
    seller = 'sell' in msg or 'buy' in msg or 'trader' in msg or 'flash' in msg
    bitcoin = 'btc' in msg
    if bitcoin and seller:
        return 'bitcoin'
    
    
    loader = 'loader' in msg
    card = 'mastercard' in msg or 'americanexpress' in msg or 'amex' in msg
    if card and loader and available:
        return 'card'
    
    
    different_payments = 'chime' in msg or 'paypal' in msg or 'zelle' in msg or 'g-pay' in msg or 'apple' in msg
    if different_payments and payment:
        return 'other payment forms'
    
    if blocking and (payment or available):
        return 'blocking available'
    
    return False


df['money_laundering'] = df['message_text'].apply(money_laundering)
temp = df[df['money_laundering'] != False]

print(temp.shape)

print(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)


(8887, 7)
(2138, 6)
(787, 6)


In [7]:
def website_developer(msg):
    
    available = 'avail' in msg or 'active' in msg or 'running' in msg or 'book' in msg or 'live' in msg or ' cc ' in msg or 'order more' in msg or 'direct center' in msg
    call = 'call' in msg or 'cll' in msg
    
    call_or_available = call and available
    
    google_ads = ('adword' in msg or 'ppc' in msg) and ('campaign' in msg or 'account' in msg or call or available)
    if google_ads:
        return 'google ads'
    
    
    if 'amazon' in msg and call_or_available:
        return 'amazon'
    

    if 'ios' in msg and call_or_available:
        return 'ios'
    
    
    if 'paypal' in msg and call_or_available:
        return 'paypal'
    
   
    if 'chime' in msg and call_or_available:
        return 'chime'
    
   
    if ('cashapp' in msg or 'cash app' in msg) and call_or_available:
        return 'cashapp'
    
    
    if ('quickbook' in msg) and call_or_available:
        return 'quickbook'
    
    
    if 'printer' in msg and call_or_available:
        return 'printer'
    
   
    if 'popup' in msg and call_or_available:
        return 'popup'
    
    
    if 'ebay' in msg and call_or_available:
        return 'ebay'
    
    
    if 'delta' in msg and call_or_available:
        return 'delta'
    
    
    if 'bsod' in msg and call_or_available:
        return 'bsod'
    
    if 'refund' in msg and call_or_available:
        return 'refund'
    
    
    if ' cc details ' in msg or 'indian cc' in msg or ' cc data ' in msg:
        return 'query for call center'
    
    
    if 'website traffic available' in msg or (call and available):
        return 'calls available'
    
    return False

df['website_developer'] = df['message_text'].apply(website_developer)
temp = df[df['website_developer'] != False]
print(temp.shape)


print(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)

(9444, 8)
(2826, 7)
(815, 7)


In [8]:
def call_center(msg):
    looking = 'looking' in msg or 'hiring' in msg or 'required' in msg or 'need' in msg
    agent = 'agent' in msg #or 'tech sale' in msg or 'technical sale' in msg
    
    if agent and looking:
        return 'need agents'
    
    return False

df['call_center'] = df['message_text'].apply(call_center)
temp = df[df['call_center'] != False]
print(temp.shape)

print(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)

(161, 9)
(104, 8)
(57, 8)


In [9]:
def victim_data(msg):
    lead = 'lead' in msg or 'data' in msg
    email = 'email' in msg and lead
    refund = 'refund' in msg and lead
    ivr = 'ivr' in msg and lead
    fresh = 'fresh' in msg and lead
    data = 'data available' in msg
    
    smtp = 'smtp' in msg
    bulk_sms = 'bulk sms' in msg
    blasting = 'email' in msg and 'blast' in msg

    if email or refund or ivr or fresh or data:
        return 'leads'
    
    if blasting or smtp or bulk_sms:
        return 'blasting'
    return False

df['victim_data'] = df['message_text'].apply(victim_data)
temp = df[df['victim_data'] != False]

print(temp.shape)

print(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)

(5224, 10)
(1153, 9)
(407, 9)


In [10]:
def toll_free_number(msg):
    available = 'available' in msg or 'avail' in msg or 'providing' in msg or 'get unlimited call' in msg
    toll_free_number = ('tfn' in msg or 'toll free' in msg or 'toll-free' in msg) and available
    did = 'did' in msg and available
    
    if toll_free_number or did:
        return True
    return False

df['toll_free_number'] = df['message_text'].apply(toll_free_number)
temp = df[df['toll_free_number'] != False]

print(temp.shape)

print(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)

(3294, 11)
(556, 10)
(250, 10)


In [11]:
def seo(msg):
    if ' seo' in msg or 'social media promotion' in msg or 'social media service' in msg or 'digital maketing' in msg:
        return 'social media promotion'
    return False
    
df['social_media_promotion'] = df['message_text'].apply(seo)
temp = df[df['social_media_promotion'] != False]

print(temp.shape)

print(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)

(304, 12)
(141, 11)
(57, 11)


In [12]:
def other(msg):
    logic = empty_post(msg) or money_laundering(msg) or website_developer(msg) or call_center(msg) or victim_data(msg) or seo(msg) or toll_free_number(msg)
        
    if not logic:
        return True
    return False
    

df['other'] = df['message_text'].apply(other)
temp = df[df['other'] == True]

print(temp.shape)

others_text = pd.DataFrame(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).index)
others_text.to_csv('others.tsv', sep='\t', header=True, index=False)

print(others_text.shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)

(21933, 13)
(7987, 1)
(2043, 12)


In [13]:
df.head()

,date,actor_id,message_text,chat group name,has_digit,empty_post,money_laundering,website_developer,call_center,victim_data,toll_free_number,social_media_promotion,other
0,"11/6/19, 11:31:28 AM]",Social media marketing 20:,‎messages and calls are end-to-end encrypted. ...,chat_12.txt,False,False,False,False,False,False,False,False,True
1,"11/6/19, 11:31:28 AM]",‎‪+91 98907 76732‬ created this group\n[11/8/1...,*🥳next week advance prediction video now avail...,chat_12.txt,True,False,False,False,False,False,False,False,True
2,"11/9/19, 10:55:40 AM]",‪+234 701 028 3792‬:,```is there any loader in this groups in need ...,chat_12.txt,False,False,False,False,False,False,False,False,True
3,"11/9/19, 11:53:46 AM]",‪+91 90413 82638‬:,*zeftron solutions* *instant payout* *most...,chat_12.txt,True,False,giftcard,amazon,False,False,False,False,False
4,"11/9/19, 12:01:33 PM]",‪+255 745 032 469‬:,‎image omitted [,chat_12.txt,False,False,False,False,False,False,False,False,True


In [14]:
df.to_csv('labelled_posts.tsv', sep='\t', header=True, index=False)